In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torchvision import models
from torch import nn, optim
import torch.nn.functional as F


In [ ]:
data_dir = 'data/SASE-FE/frames_small'
real = 'data/SASE-FE/frames_small/real'
fake = 'data/SASE-FE/frames_small/fake'

# List directories in real and fake
real_emotions = os.listdir(real)
fake_emotions = os.listdir(fake)


# real_emotions.remove('.DS_Store')
# fake_emotions.remove('.DS_Store')

# Length of real and fake emotions
print(len(real_emotions))
print(len(fake_emotions))


In [ ]:
# Generate training and validation data with pytorch

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                                        
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
test_data = datasets.ImageFolder(data_dir, transform=test_transforms)

# Define dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)



In [ ]:
# Get one batch of training images
images, labels = next(iter(trainloader))
print(images.shape)
print(labels.shape)


In [ ]:
# Load classifier nodel

# Load the pretrained model from pytorch
model = models.resnet18(pretrained=True)
print(model)


# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
classifier = nn.Sequential(nn.Linear(512, 256),
                            nn.ReLU(),
                            nn.Dropout(0.2),
                            nn.Linear(256, 2),
                            nn.LogSoftmax(dim=1))

model.fc = classifier
print(model)


# Train the classifier layers using backpropagation using the pre-trained network to get the features
# Track the loss and accuracy on the validation set to determine the best hyperparameters
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Train the classifier
epochs = 5
steps = 0
running_loss = 0
print_every = 5




In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# TODO:  Make a function for training the model and a fucntion for testing the model

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()


